In [2]:

import findspark

findspark.init("c:/spark")
# spark.stop()

In [3]:
import pyspark
from pyspark.sql import SparkSession
import os


conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.6.1')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.iceberg.type','hadoop')
        .set('spark.sql.catalog.iceberg.warehouse', 'iceberg-warehouse')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

## Run Queries
spark.sql("CREATE OR REPLACE TABLE iceberg.table1 (name STRING) USING iceberg")

spark


Spark Running


In [ ]:
# spark.sql('drop table iceberg.table2')

In [36]:
spark.sql("INSERT INTO iceberg.table1 VALUES ('Alex Merced'), ('Dipankar Mazumdar'), ('Jason Huges')")

spark.sql("SELECT * FROM iceberg.table1").show()

+-----------------+
|             name|
+-----------------+
|      Alex Merced|
|Dipankar Mazumdar|
|      Jason Huges|
+-----------------+



In [4]:
spark.sql("SHOW TBLPROPERTIES  iceberg.table1").show(truncate=False)

+-------------------------------+---------------+
|key                            |value          |
+-------------------------------+---------------+
|current-snapshot-id            |none           |
|format                         |iceberg/parquet|
|format-version                 |2              |
|write.parquet.compression-codec|zstd           |
+-------------------------------+---------------+



In [38]:
spark.sql("describe formatted  iceberg.table1").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                name|              string|   null|
|                    |                    |       |
|  # Metadata Columns|                    |       |
|            _spec_id|                 int|       |
|          _partition|            struct<>|       |
|               _file|              string|       |
|                _pos|              bigint|       |
|            _deleted|             boolean|       |
|                    |                    |       |
|# Detailed Table ...|                    |       |
|                Name|      iceberg.table1|       |
|                Type|             MANAGED|       |
|            Location|iceberg-warehouse...|       |
|            Provider|             iceberg|       |
|               Owner|         TommyShelby|       |
|    Table Properties|[current-snapshot...|       |
+-----------

In [39]:
spark.sql("describe table iceberg.table1").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    name|   string|   null|
+--------+---------+-------+



In [40]:
df = spark.read.format('CSV').option('header','true').load('movies_2482024.csv')
df.createOrReplaceTempView('temp_movies')
df = spark.sql('select * from temp_movies where year > 1000 and id > 0')
df.createOrReplaceTempView('temp_movies')
df.show()
# df.repartition(1).write.format("csv").save('movies_refined')
# spark.sql("CREATE OR REPLACE TABLE iceberg.movies (id INT, name STRING, year INT, ranking STRING) partitioned by (year) location 'iceberg-warehouse/movies' tblproperties ('table_type'='ICEBERG') ")

+---+--------------------+----+-------+
| id|               movie|year|ranking|
+---+--------------------+----+-------+
|  2|                   $|1971|    6.4|
| 11|   $1000 a Touchdown|1939|    6.7|
| 13|$21 a Day Once a ...|1941|     \N|
| 15|                 $30|1999|    7.5|
| 17|      $300 y tickets|2002|     \N|
| 21|           $5.15/Hr.|2004|     \N|
| 31|               $pent|2000|    4.3|
| 32|     $ucces Part One|1986|     \N|
| 33|             $windle|2002|    5.4|
| 34|             & frres|2000|     \N|
| 36|                '15'|2002|    6.8|
| 37|'24-25' ne vozvra...|1968|     \N|
| 38|                 '38|1987|    6.7|
| 39|                 '42|1951|     \N|
| 41|             '49-'17|1917|    5.8|
| 42|'60s Pop Rock Reu...|2004|     \N|
| 44|                 '68|1988|    5.5|
| 47|     '88 Dodge Aries|2002|     \N|
| 48| '93 jie tou ba wang|1993|     \N|
| 49|'94 du bi dao zhi...|1994|    5.7|
+---+--------------------+----+-------+
only showing top 20 rows



In [41]:
spark.sql('select * from temp_movies ').show()
spark.sql('select count(*) from temp_movies').show()

+---+--------------------+----+-------+
| id|               movie|year|ranking|
+---+--------------------+----+-------+
|  2|                   $|1971|    6.4|
| 11|   $1000 a Touchdown|1939|    6.7|
| 13|$21 a Day Once a ...|1941|     \N|
| 15|                 $30|1999|    7.5|
| 17|      $300 y tickets|2002|     \N|
| 21|           $5.15/Hr.|2004|     \N|
| 31|               $pent|2000|    4.3|
| 32|     $ucces Part One|1986|     \N|
| 33|             $windle|2002|    5.4|
| 34|             & frres|2000|     \N|
| 36|                '15'|2002|    6.8|
| 37|'24-25' ne vozvra...|1968|     \N|
| 38|                 '38|1987|    6.7|
| 39|                 '42|1951|     \N|
| 41|             '49-'17|1917|    5.8|
| 42|'60s Pop Rock Reu...|2004|     \N|
| 44|                 '68|1988|    5.5|
| 47|     '88 Dodge Aries|2002|     \N|
| 48| '93 jie tou ba wang|1993|     \N|
| 49|'94 du bi dao zhi...|1994|    5.7|
+---+--------------------+----+-------+
only showing top 20 rows

+--------+
|co

In [42]:
df = spark.read.format('CSV').option('header','true').load('directors.csv')
df.createOrReplaceTempView('temp_directors')
df.show()
# df.repartition(1).write.format("csv").save('movies_refined')
# spark.sql("CREATE OR REPLACE TABLE iceberg.movies (id INT, name STRING, year INT, ranking STRING) partitioned by (year) location 'iceberg-warehouse/movies' tblproperties ('table_type'='ICEBERG') ")

+---+------------------+-----------+
| id|        first_name|  last_name|
+---+------------------+-----------+
|  1|              Todd|          1|
|  2|               Les|12 Poissons|
|  3|           Lejaren|   a'Hiller|
|  4|              Nian|          A|
|  5|          Khairiya|  A-Mansour|
|  6|           Ricardo|   A. Solla|
|  8|Kodanda Rami Reddy|         A.|
|  9|     Nageswara Rao|         A.|
| 10|              Yuri|         A.|
| 11|             Swamy|     A.S.A.|
| 12|           Per (I)|      Aabel|
| 13|            Eivind|      Aaeng|
| 14|              Mang|        Aag|
| 15|           Sigfred|    Aagaard|
| 16|           Michael|    Aaglund|
| 17|            Safdar|        Aah|
| 18|            Astrid|      Aakra|
| 19|      Mohamad Reza|     Aalami|
| 20|             Jefri|Aalmuhammed|
| 21|         Rostislav|      Aalto|
+---+------------------+-----------+
only showing top 20 rows



In [43]:
spark.sql('Create TABLE IF NOT EXISTS iceberg.directors(id int, first_name string, last_name string) using ICEBERG TBLPROPERTIES("write.format.default"="avro")')
# spark.sql("Create TABLE IF NOT EXISTS iceberg.movies using ICEBERG partitioned by (year) AS select CAST(id as decimal(14,0)) as id, movie, CAST(year as decimal(14,0)) as year, ranking from temp_movies")
spark.sql("insert overwrite table iceberg.directors select CAST(id as decimal(14,0)) as id, first_name, last_name  from temp_directors").show()

++
||
++
++



In [44]:
spark.sql('select * from iceberg.directors').show()

+---+------------------+-----------+
| id|        first_name|  last_name|
+---+------------------+-----------+
|  1|              Todd|          1|
|  2|               Les|12 Poissons|
|  3|           Lejaren|   a'Hiller|
|  4|              Nian|          A|
|  5|          Khairiya|  A-Mansour|
|  6|           Ricardo|   A. Solla|
|  8|Kodanda Rami Reddy|         A.|
|  9|     Nageswara Rao|         A.|
| 10|              Yuri|         A.|
| 11|             Swamy|     A.S.A.|
| 12|           Per (I)|      Aabel|
| 13|            Eivind|      Aaeng|
| 14|              Mang|        Aag|
| 15|           Sigfred|    Aagaard|
| 16|           Michael|    Aaglund|
| 17|            Safdar|        Aah|
| 18|            Astrid|      Aakra|
| 19|      Mohamad Reza|     Aalami|
| 20|             Jefri|Aalmuhammed|
| 21|         Rostislav|      Aalto|
+---+------------------+-----------+
only showing top 20 rows



In [45]:
df = spark.read.format('CSV').option('header','true').load('movies_directors.csv')
df.createOrReplaceTempView('temp_movies_directors')
df.show()

+----+------+
|d_id|  m_id|
+----+------+
|   1|378879|
|   2|281325|
|   3| 30621|
|   3|304743|
|   4| 60570|
|   5| 63525|
|   6|118137|
|   8|  4860|
|   8| 18086|
|   8| 18458|
|   8| 23136|
|   8| 36169|
|   8| 58625|
|   8| 83754|
|   8| 90571|
|   8| 90573|
|   8|131879|
|   8|165214|
|   8|166374|
|   8|175886|
+----+------+
only showing top 20 rows



In [46]:
spark.sql('Create TABLE IF NOT EXISTS iceberg.movies_directors(d_id int, m_id int) using ICEBERG')
# spark.sql("Create TABLE IF NOT EXISTS iceberg.movies using ICEBERG partitioned by (year) AS select CAST(id as decimal(14,0)) as id, movie, CAST(year as decimal(14,0)) as year, ranking from temp_movies")
spark.sql("insert overwrite table iceberg.movies_directors select CAST(d_id as decimal(14,0)) as d_id, CAST(m_id as decimal(14,0)) as m_id  from temp_movies_directors").show()

++
||
++
++



In [47]:
spark.sql('select * from iceberg.movies_directors').show()

+----+------+
|d_id|  m_id|
+----+------+
|   1|378879|
|   2|281325|
|   3| 30621|
|   3|304743|
|   4| 60570|
|   5| 63525|
|   6|118137|
|   8|  4860|
|   8| 18086|
|   8| 18458|
|   8| 23136|
|   8| 36169|
|   8| 58625|
|   8| 83754|
|   8| 90571|
|   8| 90573|
|   8|131879|
|   8|165214|
|   8|166374|
|   8|175886|
+----+------+
only showing top 20 rows



In [48]:
spark.sql('Create TABLE IF NOT EXISTS iceberg.movies(id int, movie string, year int, ranking string) using ICEBERG PARTITIONED BY (year)')


DataFrame[]

In [49]:
# spark.sql("Create TABLE IF NOT EXISTS iceberg.movies using ICEBERG partitioned by (year) AS select CAST(id as decimal(14,0)) as id, movie, CAST(year as decimal(14,0)) as year, ranking from temp_movies")
spark.sql("insert overwrite table iceberg.movies select CAST(id as decimal(14,0)) as id, movie, CAST(year as decimal(14,0)) as year, ranking from temp_movies").show()

++
||
++
++



In [50]:
out_df = spark.sql('select a.*, concat(d.first_name," ",d.last_name) as director_name from (select m.movie,m.year,m.ranking,d_id from iceberg.movies m join iceberg.movies_directors l on m.id=l.m_id where year>1950 and year<2024) a join iceberg.directors d on a.d_id=d.id ')
out_df.show()

+--------------------+----+-------+-----+-------------------+
|               movie|year|ranking| d_id|      director_name|
+--------------------+----+-------+-----+-------------------+
|'Nendo no omen' y...|1961|     \N|56839|     Nobuo Nakagawa|
|1. maj 1961 godin...|1961|     \N|49421|    Pjer Majhrovski|
|1. maj 1961 godin...|1961|     \N|43018|   Hajrudin Krvavac|
|        10 juin 1944|1961|     \N|15163|      Maurice Cohen|
|     10000 malchikov|1961|     \N|59144|           I. Okada|
|     10000 malchikov|1961|     \N|10637|      Boris Buneyev|
|121 rue Blanche  ...|1961|     \N| 1029|   Quinto Albicocco|
|1964 New York Wor...|1961|     \N|11736|John (XVI) Campbell|
|               2+2=3|1961|     \N|48386|      André Lusicic|
|     5inui haebyeong|1961|     \N|41083|     Ki-duk (I) Kim|
|660124: The Story...|1961|     \N|18724|     Brian De Palma|
|  9 dnej odnogo goda|1961|    7.3|67789|       Mikhail Romm|
|      A byahme mladi|1961|     \N|88258|  Binka Zhelyazkova|
|A caval

In [ ]:
# out_df.write.format("csv").save('movies_refined')

In [51]:
x=spark.sql('SELECT * FROM iceberg.movies.files;')
x.show(truncate=False,vertical = True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 content            | 0                                                                                                                                                                
 file_path          | iceberg-warehouse/movies/data/year=1896/00000-25-58ac8b5d-3200-4b4b-be30-e9bdf2f12f57-0-00003.parquet                                                            
 file_format        | PARQUET                                                                                                                                                          
 spec_id            | 0                                                                                                                                                                
 partition          | {1896}                                                    

In [52]:
spark.sql('SELECT * FROM iceberg.movies.history;').show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-09-12 00:17:...|5624801734738711238|               null|               true|
|2024-09-12 00:20:...|7334931249918322859|5624801734738711238|               true|
|2024-09-12 09:30:...|3961187701446087615|7334931249918322859|               true|
|2024-09-12 15:35:...|6400500400627584846|3961187701446087615|               true|
+--------------------+-------------------+-------------------+-------------------+



In [53]:
x=spark.sql('SELECT * FROM iceberg.movies.snapshots;')
x.show(truncate=False,vertical = True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 committed_at  | 2024-09-12 00:17:33.241                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
spark.sql("select h.made_current_at, s.operation, h.snapshot_id, h.is_current_ancestor, s.summary['spark.app.id'] from iceberg.movies.history h join iceberg.movies.snapshots s on h.snapshot_id = s.snapshot_id order by made_current_at").show(truncate=False,vertical = True)

In [54]:
spark.sql('select * from iceberg.movies.manifests').show(truncate=False, vertical=True)
"""
Fields within partition_summaries column of the manifests table correspond to field_summary struct within manifest_list, with the following order: 
contains_null - contains_nan - lower_bound - upper_bound
Note: contains_nan could return null, which indicates that this information is not available from files metadata
"""

-RECORD 0-----------------------------------------------------------------------------------------------------
 content                     | 0                                                                              
 path                        | iceberg-warehouse/movies/metadata/f93fccca-3537-48be-afa9-65d53e0f1c43-m1.avro 
 length                      | 15921                                                                          
 partition_spec_id           | 0                                                                              
 added_snapshot_id           | 6400500400627584846                                                            
 added_data_files_count      | 122                                                                            
 existing_data_files_count   | 0                                                                              
 deleted_data_files_count    | 0                                                                              
 

'\nFields within partition_summaries column of the manifests table correspond to field_summary struct within manifest_list, with the following order: \ncontains_null - contains_nan - lower_bound - upper_bound\nNote: contains_nan could return null, which indicates that this information is not available from files metadata\n'

In [55]:
spark.sql("select * from iceberg.movies where id>2000 and id<300000").show()

+-----+--------------------+----+-------+
|   id|               movie|year|ranking|
+-----+--------------------+----+-------+
|12718|Alumnos de Chapul...|1896|     \N|
|12719|Alumnos de Chapul...|1896|     \N|
|13895|American Falls fr...|1896|     \N|
|15226|          Amy Muller|1896|    4.3|
|17624|Annabelle in Flag...|1896|     \N|
|19429|            Aquarium|1896|     \N|
|20816|Arrive d'un train...|1896|     \N|
|20817|Arrive d'un train...|1896|     \N|
|20820|Arrive d'un train...|1896|     \N|
|21588|Asalto a sable de...|1896|     \N|
|23212|Atlantic City Bat...|1896|     \N|
|23215|Atlantic City Boa...|1896|     \N|
|23216|Atlantic City Boa...|1896|     \N|
|24687|           Autruches|1896|    5.4|
|24981|Avenida da Liberdade|1896|     \N|
|25396|    Awakening of Rip|1896|     \N|
|25693|  Azenhas no Rio Ave|1896|     \N|
|26189|      Babies Quarrel|1896|    6.4|
|27835|     Baignade en mer|1896|     \N|
|27862|Baile de la romer...|1896|     \N|
+-----+--------------------+----+-

In [56]:
spark.sql('show partitions iceberg.movies').show()
#spark UI
#hive
# In Apache Hive, the files in a partition are scanned at runtime, while in Iceberg there is a manifest file which improves performance.

AnalysisException: Table iceberg.movies does not support partition management.; line 1 pos 16;
ShowPartitions [partition#1485]
+- ResolvedTable org.apache.iceberg.spark.SparkCatalog@76531dad, movies, iceberg.movies, [id#1486, movie#1487, year#1488, ranking#1489]


In [58]:
df = spark.read.format('CSV').option('header','true').load('movies_2482024.csv')
df.createOrReplaceTempView('temp_movies_r')
df = spark.sql('select * from temp_movies_r where year >= 1995 and year <= 2000')
df.createOrReplaceTempView('temp_movies_r')
df.show()
spark.sql("insert overwrite table iceberg.movies select CAST(id as decimal(14,0)) as id, movie, CAST(year as decimal(14,0)) as year, ranking from temp_movies_r").show()

+---+--------------------+----+-------+
| id|               movie|year|ranking|
+---+--------------------+----+-------+
| 15|                 $30|1999|    7.5|
| 31|               $pent|2000|    4.3|
| 34|             & frres|2000|     \N|
| 64|       'Bats' Abound|1999|     \N|
| 66|'Betty Bee' (sopr...|1999|     \N|
|101|'El Chicko' - der...|1995|    4.1|
|118|'Halloween' Unmas...|1999|     \N|
|136|'Java Madness' fo...|1995|    6.6|
|145|'Ks-Bels' letmdja...|1995|     \N|
|152|            'M' Word|1996|     \N|
|155|'Man Who Couldn't...|1996|     \N|
|176|'N Sync & Britney...|2000|     \N|
|177|  'N Sync In Concert|1998|     \N|
|178|'N Sync: 'Ntimate...|2000|     \N|
|179|'N Sync: Live Fro...|2000|     \N|
|180|'N Sync: Making t...|2000|     \N|
|181|'N Sync: No Strin...|1999|     \N|
|184|'Ne gnstige Geleg...|1999|    4.6|
|199|         'O' de amor|1995|     \N|
|201|'On the Inside: C...|2000|     \N|
+---+--------------------+----+-------+
only showing top 20 rows

++
||
++
++



In [59]:
spark.sql('select count(*) from iceberg.movies').show(truncate=False, vertical=True)

-RECORD 0---------
 count(1) | 48230 



In [60]:
spark.sql('select count(*) from iceberg.movies for SYSTEM_TIME as of "2024-09-12 00:19:54.899"').show(truncate=False, vertical=True)

-RECORD 0----------
 count(1) | 302134 



In [61]:
spark.sql('select count(*) from iceberg.movies for SYSTEM_VERSION as of "5624801734738711238"').show(truncate=False, vertical=True)

-RECORD 0----------
 count(1) | 302134 



In [ ]:
spark.sql('select count(*) from iceberg.movies for SYSTEM_VERSION as of "7334931249918322859"').show(truncate=False, vertical=True)

In [ ]:
spark.sql('select count(*) from iceberg.movies for SYSTEM_VERSION as of "3651939683099648566"').show(truncate=False, vertical=True)

In [ ]:
spark.sql('select count(*) from iceberg.movies for SYSTEM_VERSION as of "8158436517200082356"').show(truncate=False, vertical=True)

In [ ]:
spark.read.option("snapshot-id", 7334931249918322859).table("iceberg.movies").show()

In [62]:
spark.sql("ROLLBACK TABLE iceberg.movies TO SNAPSHOT '3961187701446087615'")
# spark.sql("CALL iceberg.system.rollback_to_snapshot('iceberg.movies', '6687819643318083920')")
# spark.sql("ALTER TABLE iceberg.movies EXECUTE ROLLBACK(3651939683099648566);")

ParseException: 
Operation not allowed: ROLLBACK.(line 1, pos 0)

== SQL ==
ROLLBACK TABLE iceberg.movies TO SNAPSHOT '3961187701446087615'
^^^
